In [1]:
using Pkg
Pkg.add("XLSX")
import XLSX

#Load Oscar
using Oscar

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed snappy_jll ─ v1.1.9+1
  No Changes to `~/.julia/environments/v1.6/Project.toml`
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [fe1e1685] ↑ snappy_jll v1.1.9+0 ⇒ v1.1.9+1
Precompiling project...
  ✓ snappy_jll
  ✓ MongoC_jll
  ✓ Mongoc
  ✓ Polymake
  ✓ Oscar
  5 dependencies successfully precompiled in 56 seconds (79 already precompiled)


 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.8.2 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


<font size="5">
In the first step the variables and the corresponding polynomial ring are defined. The coefficients of the polynomials can either be zero or one only (from the 2-elements field). 
</font>

In [2]:
R1 = ResidueRing(ZZ, 2);
myR, (A, h, H, a, x, X) = PolynomialRing(R1, ["A", "h", "H", "a", "x", "X"]);
myvars=[A, h, H, a, x, X];

<font size="5">
Now your xlsx-table is loaded. The table consists of entries 0 and 1.
The array 'indices' is also loaded. It includes an own labeling of the objects in the table (if wanted).
</font>

In [8]:
chartable=XLSX.readdata("../Data/Karnak_choice.xlsx", "Analysis_09_02_22", "A2:F496");
indices=1:495;

<font size="5">
The analysis can be restricted to a subset of the characteristics. Select some of them.
</font>

In [9]:
varnum=6; choice=1:6

1:6

<font size="5">
According to your choice, the Boolean entries are transformed into an array of polynomials.
</font>

In [10]:
chartable=chartable[:,choice];
myvars=myvars[choice];
expr_list=prod(myvars+ myvars.^0 -chartable[1,:]); 
for i=2:size(chartable,1)
    expr_list=vcat(expr_list, prod(myvars+ myvars.^0 -chartable[i,:])); 
end

<font size="5">
For the weighting of the monomials, we do some statistics about the marginal distribution of characteristics
</font>

In [11]:
monweight=ones(Int,varnum);
for i=1:size(myvars,1)
    print(myvars[i]);
    print(" true=");
    print(sum(chartable[:,i]));
    print(" false=");
    print(size(chartable,1)-sum(chartable[:,i]));
    print("\n");
    monweight[i]=sum(chartable[:,i])*(size(chartable,1)-sum(chartable[:,i]));
end
monweight

A true=170 false=325
h true=94 false=401
H true=58 false=437
a true=203 false=292
x true=152 false=343
X true=17 false=478


6-element Vector{Int64}:
 55250
 37694
 25346
 59276
 52136
  8126

<font size="5">
The algorithm generates an ideal in the set of Boolean Polynomials. This ideal represents every combination of characteristics which is not to be found in your table. The Gröbner Basis of this ideal is computed.
</font>

In [14]:
uq_expr_list=unique(expr_list);
expr=sum(uq_expr_list)+1;
generator=myvars.^2+myvars; II=ideal(myR, vcat(generator, expr));
Y, m = quo(myR, II);
GB = groebner_basis(II, ordering = wdeglex(gens(myR),monweight))

16-element Vector{nmod_mpoly}:
 X^2 + X
 h*X + H*X
 H^2 + H
 h^2 + h
 A*H*X + H*X
 H*a*X + H*x*X
 x^2 + x
 A^2 + A
 h*H*x + H*x
 A*x*X + x*X
 A*h*H + A*H + h*H + H
 a^2 + a
 h*H*a + H*a
 A*H*x + H*x
 A*H*a + A*H + h*H*a + H
 A*h*H*a + A*h*H*x + A*h*H + A*h*a*x + A*h*x + h*H*a + h*H*x + h*H + h*a*x + h*x

<font size="5">
According to the "rules" included in the Gröbner Basis the simplification can be applied to all entries in the loaded table:
</font>

In [13]:
for i = 1:size(uq_expr_list,1)
    res=normal_form(uq_expr_list[i],GB);
    counter=0;
    for j = 1:size(expr_list,1)
        if (uq_expr_list[i]==expr_list[j])
            counter=counter+1;
            printstyled(indices[j], color=:red);
            print(" ");
        end
    end
    printstyled(" (#", color=:red);
    printstyled(counter, color=:red);
    printstyled(")", color=:red);
    print("\n");
    printstyled(factor(uq_expr_list[i]), color=:green);
    wrtnorm=0;
    if (wrtnorm==1)
        printstyled("=\n", color=:green);
        printstyled(uq_expr_list[i], color=:green);
    end
    print("\n");
    printstyled(res, bold=:true);
    print("\n\n");
end

LoadError: UndefVarError: GB not defined